<a href="https://colab.research.google.com/github/khadijabendib/Agent-Intelligent/blob/main/Robotique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

La création de l'environnement avec des murs et la position cible 

In [92]:
import numpy as np

# Définir la taille de l'environnement
n_rows = 20
n_cols = 20

matrice = np.zeros((n_rows, n_cols), dtype=int)

# Ajouter les murs
matrice[0, :] = -1
matrice[-1, :] = -1
matrice[:, 0] = -1
matrice[:,9] = -1
matrice[9,:] = -1
matrice[9,7] = 0
matrice[9,16] = 0
matrice[5,9] = 0
matrice[16,9] = 0
matrice[:, -1] = -1

# Définir la position attendue
goal_pos = (15, 17)
matrice[goal_pos] = 10
print(matrice)

[[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0 -1]
 [-1  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0 -1]
 [-1  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0 -1]
 [-1  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0 -1]
 [-1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1]
 [-1  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0 -1]
 [-1  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0 -1]
 [-1  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0 -1]
 [-1 -1 -1 -1 -1 -1 -1  0 -1 -1 -1 -1 -1 -1 -1 -1  0 -1 -1 -1]
 [-1  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0 -1]
 [-1  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0 -1]
 [-1  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0 -1]
 [-1  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0 -1]
 [-1  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0 -1]
 [-1  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0 10

Cette matrice est utilisée pour stocker les valeurs Q pour chaque état-action de l'environnement.

In [88]:
q_values = np.zeros((n_rows, n_cols, 4))

La liste actions contient les différentes actions possibles dans un environnement:
#'up' : action de déplacement vers le haut.
#'right' : action de déplacement vers la droite.
#'down' : action de déplacement vers le bas.
#'left' : action de déplacement vers la gauche.

In [89]:
actions=['up','right','down','left']

Ce code définit une fonction appelée "is_terminal_state" qui prend en argument les indices de la ligne et de la colonne d'une certaine position. La fonction vérifie si l'état correspondant à cette position est un état terminal.



In [90]:
#define a function that determines if the specified location is a terminal state
def is_terminal_state(current_row_index, current_column_index):
  #if the reward for this location is 0
  if matrice[current_row_index, current_column_index] == 0 :
    return False
  else:
    return True

Le code présent contient une fonction appelée "get_starting_location" qui joue un rôle essentiel dans la sélection d'une position de départ pour une partie ou une simulation. Cette fonction utilise la méthode de sélection aléatoire pour choisir une position de départ dans une matrice ou un environnement. Elle commence par générer des indices de ligne et de colonne aléatoires en utilisant la fonction "np.random.randint" en fonction des dimensions de la matrice ou de l'environnement. Ensuite, la fonction vérifie si la position générée aléatoirement est un état terminal en appelant la fonction "is_terminal_state". Si la position est un état terminal, la fonction génère de nouveaux indices aléatoires jusqu'à ce qu'une position non terminale soit trouvée. Une fois qu'une position non terminale est obtenue, les indices de ligne et de colonne correspondants sont renvoyés en tant que tuple, prêts à être utilisés comme position de départ valide pour le jeu ou la simulation. En intégrant cette fonction, l'application garantit que chaque partie ou simulation commence dans une position non terminale, offrant ainsi une expérience de jeu cohérente et évitant les situations indésirables où le jeu se termine immédiatement.

In [91]:
#define a function that will choose a random, non-terminal starting location
def get_starting_location():
  #get a random row and column index
  current_row_index = np.random.randint(n_rows)
  current_column_index = np.random.randint(n_cols)

  while is_terminal_state(current_row_index, current_column_index):
    current_row_index = np.random.randint(n_rows)
    current_column_index = np.random.randint(n_cols)
  return current_row_index, current_column_index

La fonction "get_next_action" joue un rôle essentiel dans le processus de sélection de la prochaine action à prendre dans un environnement d'apprentissage par renforcement. Cette fonction utilise une stratégie d'exploration-exploitation pour équilibrer l'exploration de nouvelles actions et l'exploitation des connaissances déjà acquises. Elle prend en compte le paramètre epsilon qui détermine le niveau d'exploration. Si une valeur aléatoire générée entre 0 et 1 est inférieure à epsilon, la fonction choisit la meilleure action en se basant sur les valeurs Q présentes dans la table "q_values" pour l'état actuel. Cela est réalisé en utilisant la fonction "np.argmax" pour sélectionner l'indice de l'action ayant la valeur Q maximale. En revanche, si la valeur aléatoire est supérieure ou égale à epsilon, la fonction choisit une action aléatoire parmi les actions possibles. L'intégration de cette fonction dans l'algorithme d'apprentissage par renforcement permet d'optimiser la prise de décision en exploitant les connaissances acquises tout en explorant de nouvelles actions pour améliorer les performances globales du système.

In [76]:
def get_next_action(current_row_index, current_column_index, epsilon):
  #if a randomly chosen value between 0 and 1 is less than epsilon, 
  #then choose the most promising value from the Q-table for this state.
  if np.random.random() < epsilon:
    # print("hhhhhh")
    return np.argmax(q_values[current_row_index, current_column_index])
  else: #choose a random action
    return np.random.randint(4)

La fonction "get_next_location" est essentielle dans la détermination de la prochaine position en fonction de l'action choisie dans un environnement de jeu ou une simulation. Cette fonction prend en compte les indices actuels de ligne et de colonne de la position courante, ainsi que l'indice de l'action choisie. En fonction de l'action sélectionnée, la fonction ajuste les indices de ligne et de colonne de manière appropriée pour calculer la nouvelle position.



In [78]:
#define a function that will get the next location based on the chosen action
def get_next_location(current_row_index, current_column_index, action_index):
  new_row_index = current_row_index
  new_column_index = current_column_index
  if actions[action_index] == 'up' and current_row_index > 0:
    new_row_index -= 1
  elif actions[action_index] == 'right' and current_column_index < n_cols - 1:
    new_column_index += 1
  elif actions[action_index] == 'down' and current_row_index < n_rows - 1:
    new_row_index += 1
  elif actions[action_index] == 'left' and current_column_index > 0:
    new_column_index -= 1
  return new_row_index, new_column_index


La fonction "get_shortest_path" est une composante essentielle permettant d'obtenir le chemin le plus court entre n'importe quelle position autorisée dans l'entrepôt où le robot peut se déplacer et la position de l'emballage des articles. Elle utilise des fonctions auxiliaires telles que "is_terminal_state", "get_next_action" et "get_next_location" pour déterminer les actions à prendre et les positions successives du chemin.

In [79]:
#Define a function that will get the shortest path between any location within the warehouse that 
#the robot is allowed to travel and the item packaging location.
def get_shortest_path(start_row_index, start_column_index):
  #return immediately if this is an invalid starting location
  if is_terminal_state(start_row_index, start_column_index):
    return []
  else: #if this is a 'legal' starting location
    current_row_index, current_column_index = start_row_index, start_column_index
    shortest_path = []
    shortest_path.append([current_row_index, current_column_index])
    #continue moving along the path until we reach the goal (i.e., the item packaging location)
    while not is_terminal_state(current_row_index, current_column_index):
      #get the best action to take
      action_index = get_next_action(current_row_index, current_column_index, 1.)
      #move to the next location on the path, and add the new location to the list
      current_row_index, current_column_index = get_next_location(current_row_index, current_column_index, action_index)
      shortest_path.append([current_row_index, current_column_index])
      print("["+ str(current_row_index)  +", "+ str(current_column_index) +"]")
    return shortest_path

Création d'une représentation d'un environnement dans une matrice afin de pouvoir effectuer des opérations et des calculs sur cet environnement. Cela peut être utilisé pour développer des algorithmes de déplacement, de planification de chemin ou d'exploration dans cet environnement simulé.

In [ ]:
#define training parameters
epsilon = 0.9 #the percentage of time when we should take the best action (instead of a random action)
discount_factor = 0.9 #discount factor for future rewards
learning_rate = 0.9 #the rate at which the AI agent should learn

#run through 1000 training episodes
for episode in range(100000):
  #get the starting location for this episode
  row_index, column_index = get_starting_location()
  print(episode)
  #continue taking actions (i.e., moving) until we reach a terminal state
  #(i.e., until we reach the item packaging area or crash into an item storage location)
  while not is_terminal_state(row_index, column_index):
    #choose which action to take (i.e., where to move next)
    action_index = get_next_action(row_index, column_index, epsilon)
    # print(action_index)
    #perform the chosen action, and transition to the next state (i.e., move to the next location)
    old_row_index, old_column_index = row_index, column_index #store the old row and column indexes
    row_index, column_index = get_next_location(row_index, column_index, action_index)
    
    #receive the reward for moving to the new state, and calculate the temporal difference
    reward = matrice[row_index, column_index]
    # print(reward)
    old_q_value = q_values[old_row_index, old_column_index, action_index]
    temporal_difference = reward + (discount_factor * np.max(q_values[row_index, column_index])) - old_q_value

    #update the Q-value for the previous state and action pair
    new_q_value = old_q_value + (learning_rate * temporal_difference)
    q_values[old_row_index, old_column_index, action_index] = new_q_value

print('Training complete!')

In [82]:
print(matrice)

[[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0 -1]
 [-1  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0 -1]
 [-1  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0 -1]
 [-1  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0 -1]
 [-1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1]
 [-1  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0 -1]
 [-1  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0 -1]
 [-1  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0 -1]
 [-1 -1 -1 -1 -1 -1 -1  0 -1 -1 -1 -1 -1 -1 -1 -1  0 -1 -1 -1]
 [-1  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0 -1]
 [-1  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0 -1]
 [-1  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0 -1]
 [-1  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0 -1]
 [-1  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0 -1]
 [-1  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0 10

In [83]:
#display a few shortest paths
print(get_shortest_path(16, 16)) #starting at row 16, column 16
# print(get_shortest_path(5, 0)) #starting at row 5, column 0
print(get_shortest_path(17, 17)) #starting at row 17, column 17

[15, 16]
[15, 17]
[[16, 16], [15, 16], [15, 17]]
[16, 17]
[15, 17]
[[17, 17], [16, 17], [15, 17]]
